# Abgabe 3

In [1]:
import numpy as np

In [ ]:
#load images
images = []
for file in os.listdir('.'):
    if file.endswith(('.jpg')):
        img = io.imread(file)
        images.append(img)

plt.imshow(images[0])
plt.axis('off') 
plt.show()

#convert to gray image 
def gray_converter(image):
    gray_image = 0.2989 * image[:, :, 2] + 0.5870 * image[:, :, 1] + 0.1140 * image[:, :, 0]
    gray_image = gray_image.astype('uint8')
    return gray_image

2. Implementierung der Faltung mit Filtermasken

In [ ]:

#smoothing filter
kernel = np.array([[0.075, 0.125, 0.075],
                   [0.125, 0.200, 0.125],
                   [0.075, 0.125, 0.075]], dtype= float)


def filter(in_image, filter, off):

    image_h, image_w = in_image.shape
    filter_h, filter_w = filter.shape

    s = 1.0 / np.sum(filter)

    #filter matrix (2K+1)×(2L+1)
    K = filter.shape[0] // 2
    L = filter.shape[1] //2

    copy = in_image.copy()
    out_image = np.zeros((image_h - 2 * L, image_w - 2 * K), dtype=np.uint8) # -2 border pixel

    for v in range(L, image_h - L, off):
        for u in range(K, image_w - K, off):
            sum = 0.0
            for j in range(- L, L + 1):
                for i in range(- K, K + 1):
                    p = copy[v + j, u + i] #[y,x]
                    c = filter[j + L][i + K]
                    sum += c * p
            
            q = int(np.round(s * sum))
            if q < 0: q = 0
            if q > 255: q = 255
            out_image[v, u] = q
        
    return out_image